<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# AWS - Daily Billing Notification to Slack
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/AWS/AWS_Daily_biling_notification_to_slack.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a>

#AWS #Billing #Notification #Slack

## Input

In [ ]:
pip install boto3

### Library

In [ ]:
import datetime
import boto3
import naas
import dateutil.relativedelta
import pandas as pd
import naas_drivers

### Variables

In [ ]:
# AWS account
AWS_ACCESS_KEY_ID = "***"
AWS_SECRET_ACCESS_KEY = "***"

# Slack
SLACK_TOKEN = "***"
SLACK_CHANNEL = "-aws-billing"

### Constants

In [ ]:
def last_day_of_month(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

today = datetime.date.today()
lastDay = last_day_of_month(today)
start_month_date = (today - dateutil.relativedelta.relativedelta(months=12))

start_date = "{}-{:02d}-{:02d}".format(today.year, today.month, 1)
end_date = "{}-{:02d}-{:02d}".format(today.year, today.month, today.day)
last_day = "{}-{:02d}-{:02d}".format(lastDay.year, lastDay.month, lastDay.day)

### Connect to AWS

In [ ]:
client = boto3.client('ce',
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

## Model

### Get current cost from AWS billing

In [ ]:
result = client.get_cost_and_usage(
    TimePeriod = {
        'Start': start_date,
        'End': end_date
    },
    Granularity = 'MONTHLY',
    Filter = {
        "Dimensions": {
                    "Key": "RECORD_TYPE",
                    "Values": ["Credit", "Refund"]
                }
    },
    Metrics = ["BlendedCost"],
    GroupBy = [
        {
            'Type': 'DIMENSION',
            'Key': 'SERVICE'
        },
        {
            'Type': 'DIMENSION',
            'Key': 'USAGE_TYPE'
        }
    ]
)

### Transform current billing to dataframe

In [ ]:
df_billing = pd.DataFrame()

for t in result["ResultsByTime"]:
    for r in t["Groups"]:
        dimension = r["Keys"][0]
        usage_type = r["Keys"][1]
        amount = r["Metrics"]["BlendedCost"]["Amount"]
        period_start = t["TimePeriod"]["Start"]
        period_end = t["TimePeriod"]["End"]
        df_billing = df_billing.append({
            "Dimension": dimension,
            "UsageType": usage_type,
            "Amount": amount,
            "PeriodStart": period_start,
            "PeriodEnd": period_end
        }, ignore_index=True)
df_billing = df_billing.astype({'Amount': 'float'})

# Display result
df_billing.tail(5)

### Get forecast from AWS

In [ ]:
ce_forecast = client.get_cost_forecast(
    TimePeriod={
        'Start': end_date,
        'End': last_day
    },
    Metric='BLENDED_COST',
    Granularity='MONTHLY'
)

## Output

### Save current billing to csv

In [ ]:
df_billing.to_csv('monthly.csv')
naas.asset.add(path='monthly.csv')

### Data from AWS billing

In [ ]:
current_amount = df_billing["Amount"].sum()
forecast = float(ce_forecast["Total"]["Amount"])
asset_link = "<Copy/Paste the asset link from the previous cell 👆>"

### Create message for slack

In [ ]:
message = """
👋 Daily AWS Billing notification

Current spending: [*{:.2f}$*]
Forecast: [*{:.2f}$*]
End of month estimate: [*{:.2f}$*]

You can download the details here 👇 {}
""".format(float(current_amount), float(forecast), float(current_amount - forecast), asset_link)
image_url = None # Set to None if you don't need it
print(message)

### Send data to slack

In [ ]:
try:
    naas_drivers.slack.connect(SLACK_TOKEN).send(SLACK_CHANNEL, message)
except:
    pass

### Scheduler

In [ ]:
naas.scheduler.add(cron="0 9 * * *")